### 1) Word concreteness
XLSX: Word	Bigram	Conc.M	Conc.SD	Unknown	Total	Percent_known	SUBTLEX  
->  
CSV: word, conc  
where conc is normalized Conc.M

### 2) Multiword expression concreteness
CSV: Expression, Mean_C, SD_C, N_Rate, N_Unknown  
->  
CSV: expression, conc, n_words  
where conc is normalized Mean_C  
rows with NA in Mean_C are dropped

In [ ]:
import pandas as pd
from utils import IMAG_DATA_DIR

MWE_INPUT_PATH = IMAG_DATA_DIR / "raw" / "multiword.csv"
MWE_OUTPUT_PATH = IMAG_DATA_DIR / "multiword.csv"

mwe_df = pd.read_csv(MWE_INPUT_PATH)

mwe_csv_df = mwe_df[["Expression", "Mean_C"]].rename(
    columns={"Expression": "expression", "Mean_C": "conc"}
)
mwe_csv_df = mwe_csv_df.dropna().reset_index(drop=True)

# Normalize 'conc' column (min-max normalization)
mwe_csv_df["conc"] = (mwe_csv_df["conc"] - mwe_csv_df["conc"].min()) / (
    mwe_csv_df["conc"].max() - mwe_csv_df["conc"].min()
)

INPUT_PATH = IMAG_DATA_DIR / "raw" / "words.xlsx"
OUTPUT_PATH = IMAG_DATA_DIR / "words.csv"

xlsx_df = pd.read_excel(INPUT_PATH)
csv_df = xlsx_df[["Word", "Conc.M", "Bigram"]].rename(
    columns={"Word": "word", "Conc.M": "conc", "Bigram": "is_bigram"}
)
csv_df = csv_df.dropna().reset_index(drop=True)

# Normalize 'conc' column (min-max normalization)
csv_df["conc"] = (csv_df["conc"] - csv_df["conc"].min()) / (
    csv_df["conc"].max() - csv_df["conc"].min()
)

# Average scores of words contained in both datasets into expressions
duplicate_df = csv_df[csv_df["word"].isin(mwe_csv_df["expression"])]
merged = duplicate_df.merge(
    mwe_csv_df[["expression", "conc"]],
    left_on="word",
    right_on="expression",
    suffixes=("_word", "_expr"),
)
avg_conc = (merged["conc_word"] + merged["conc_expr"]) / 2
mwe_csv_df.loc[mwe_csv_df["expression"].isin(merged["expression"]), "conc"] = (
    avg_conc.values
)
csv_df = csv_df[~csv_df["word"].isin(mwe_csv_df["expression"])].reset_index(drop=True)
print(
    f"Averaged {len(duplicate_df)} words contained in both single and multi-word expressions to expressions and removed them from single words"
)

# Move bigrams from csv_df to mwe_csv_df
bigrams_df = csv_df[csv_df["is_bigram"] == 1].copy()
bigrams_df = bigrams_df[["word", "conc"]].rename(columns={"word": "expression"})
mwe_csv_df = pd.concat([mwe_csv_df, bigrams_df], ignore_index=True)
csv_df = csv_df[csv_df["is_bigram"] == 0].reset_index(drop=True)
csv_df = csv_df[["word", "conc"]]
print(f"Moved {len(bigrams_df)} bigrams to multiword expressions")

csv_df.to_csv(OUTPUT_PATH, index=False)

# Add n_words column
mwe_csv_df["n_words"] = mwe_csv_df["expression"].str.split().str.len()

mwe_csv_df.to_csv(MWE_OUTPUT_PATH, index=False)

Averaged 2928 words contained in both single and multi-word expressions to expressions and removed them from single words
Moved 68 bigrams to multiword expressions


### 3) Preposition imageability
XLSX: Form	NewIm	SD	Raters	Glossed	Concr	PubIm	PubImC	is.imrat	COCA.freq	Nmorphs	Preval	Joy  
->  
CSV: prep, imag  
where imag is normalized NewIm

Problems:
- outside [pos_=ADP]
- on.(loc) [pos_=ADP] vs on.(adv) [pos_!=ADP]
- to [pos_=ADP] vs to(inf) [pos_=PART]

In [ ]:
import pandas as pd
from utils import IMAG_DATA_DIR
from IPython.display import display

INPUT_PATH = IMAG_DATA_DIR / "raw" / "prepositions.xlsx"
OUTPUT_PATH = IMAG_DATA_DIR / "prepositions.csv"

prep_df = pd.read_excel(INPUT_PATH)
prep_csv_df = prep_df[["Form", "NewIm"]].rename(
    columns={"Form": "prep", "NewIm": "imag"}
)
prep_csv_df = prep_csv_df.dropna().reset_index(drop=True)

TO_REMOVE = ["on.the.oth.side(of)", "diag.opposite", "by(“walk.by”)"]
prep_csv_df = prep_csv_df[~prep_csv_df["prep"].isin(TO_REMOVE)].reset_index(drop=True)

# Assign the average imageability of "in.among(st)" and "among" to "in.among(st)" and remove "among" row and rename "in.among(st)" to "among(st)"
among_imag = (
    prep_csv_df[prep_csv_df["prep"] == "in.among(st)"]["imag"].values[0]
    + prep_csv_df[prep_csv_df["prep"] == "among"]["imag"].values[0]
) / 2
prep_csv_df.loc[prep_csv_df["prep"] == "in.among(st)", "imag"] = among_imag
prep_csv_df = prep_csv_df[prep_csv_df["prep"] != "among"].reset_index(drop=True)
prep_csv_df.loc[prep_csv_df["prep"] == "in.among(st)", "prep"] = "among(st)"

# Replace . with space in preposition names
prep_csv_df["prep"] = (
    prep_csv_df["prep"]
    .str.replace(".", " ", regex=False)
    .str.replace("(of)", "", regex=False)
)

S_SUFFIX = [
    "backward(s)",
    "forward(s)",
    "among(st)",
    "upward(s)",
    "toward(s)",
    "onward(s)",
    "outward(s)",
    "inward(s)",
]

# Convert the words with suffixes and [sub-, over-, inter-] to regex patterns, add regex flag column with 1 for these rows
prep_csv_df["is_regex"] = prep_csv_df["prep"].apply(
    lambda x: 1 if x in [*S_SUFFIX, "sub-", "over-", "inter-"] else 0
)
# Replace [sub-, over-, inter-] with [sub\w+, over\w+, inter\w+]
prep_csv_df["prep"] = prep_csv_df["prep"].apply(
    lambda x: x.replace("sub-", "sub\\w+")
    .replace("over-", "over\\w+")
    .replace("inter-", "inter\\w+")
    if x in ["sub-", "over-", "inter-"]
    else x
)
# Convert suffixes to regex patterns with s/st optional
prep_csv_df["prep"] = prep_csv_df["prep"].apply(
    lambda x: x.replace("(st)", "(s|st)?")
    if x.endswith("(st)")
    else x.replace("(s)", "(s)?")
    if x.endswith("(s)")
    else x
)

# Normalize 'imag' column (min-max normalization)
prep_csv_df["imag"] = (prep_csv_df["imag"] - prep_csv_df["imag"].min()) / (
    prep_csv_df["imag"].max() - prep_csv_df["imag"].min()
)

# Add pos_adp and pos_nonadp columns with NA values
prep_csv_df["pos_adp"] = "NA"
prep_csv_df["pos_nonadp"] = "NA"

prep_csv_df.loc[prep_csv_df["prep"] == "outside", "pos_adp"] = prep_csv_df[
    prep_csv_df["prep"] == "outside"
]["imag"].values[0]

# Convert "on (loc) and on (adv)" to "on" with pos_adp=<imag from "on (loc)"> and pos_nonadp=<imag from "on (adv)">
loc_imag = prep_csv_df[prep_csv_df["prep"] == "on (loc)"]["imag"].values[0]
adv_imag = prep_csv_df[prep_csv_df["prep"] == "on (adv)"]["imag"].values[0]
prep_csv_df = prep_csv_df[prep_csv_df["prep"] != "on (adv)"].reset_index(drop=True)
prep_csv_df.loc[prep_csv_df["prep"] == "on (loc)", "prep"] = "on"
prep_csv_df.loc[prep_csv_df["prep"] == "on", "pos_adp"] = loc_imag
prep_csv_df.loc[prep_csv_df["prep"] == "on", "pos_nonadp"] = adv_imag

to_inf_imag = prep_csv_df[prep_csv_df["prep"] == "to(inf)"]["imag"].values[0]
prep_csv_df = prep_csv_df[prep_csv_df["prep"] != "to(inf)"].reset_index(drop=True)
prep_csv_df.loc[prep_csv_df["prep"] == "to", "pos_adp"] = prep_csv_df[
    prep_csv_df["prep"] == "to"
]["imag"].values[0]
prep_csv_df.loc[prep_csv_df["prep"] == "to", "pos_nonadp"] = to_inf_imag

# Add n_words column
prep_csv_df["n_words"] = prep_csv_df["prep"].str.split().str.len()

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(prep_csv_df)

prep_csv_df.to_csv(OUTPUT_PATH, index=False)

,prep,imag,is_regex,pos_adp,pos_nonadp,n_words
0,on top,1.000000,0,NA,NA,2
1,aboard,0.996868,0,NA,NA,1
2,behind,0.990034,0,NA,NA,1
3,in between,0.983485,0,NA,NA,2
4,across from,0.976651,0,NA,NA,2
5,close to,0.974374,0,NA,NA,2
6,beneath,0.969533,0,NA,NA,1
7,up,0.961560,0,NA,NA,1
8,underneath,0.939920,0,NA,NA,1
9,above,0.911162,0,NA,NA,1


### Places dataset
http://places2.csail.mit.edu/index.html  
TXT: /s/swimming_pool/indoor 325  
->  
TXT: swimming pool

In [4]:
from utils import IMAG_DATA_DIR

PLACES_ORIG_PATH = IMAG_DATA_DIR / "raw" / "places365.txt"
PLACES_OUTPUT_PATH = IMAG_DATA_DIR / "places.txt"

with open(PLACES_ORIG_PATH, "r") as f:
    lines = f.readlines()

with open(PLACES_OUTPUT_PATH, "w") as f:
    places = [line.strip().split()[0].split("/")[2].replace("_", " ") for line in lines]
    # Deduplicate
    places = list(set(places))
    places.sort()
    f.writelines(place + "\n" for place in places)